In [168]:
using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using MeshCat
using MeshCatMechanisms
using MechanismGeometries
using CoordinateTransformations
using GeometryBasics

include("/home/hkolano/onr-dynamics-julia/simulate_with_ext_forces.jl")
println("Libraries imported.")

Libraries imported.


In [146]:
# model definition
# ------------------------------------------------------------------------
#                            MODEL DEFINITION
# ------------------------------------------------------------------------
vis = Visualizer()
render(vis)
# Define paths to the URDF files
src_dir = dirname(pathof(RigidBodyDynamics))
urdf_file = joinpath(src_dir, "..", "..", "..", "..", "..", "onr-dynamics-julia", "URDFs", "alphaArmCombo.urdf")
mechanism_alpha = parse_urdf(urdf_file; gravity = [0.0, 0.0, 0.0])

delete!(vis)

# Create visuals of the URDFs
visuals = URDFVisuals(urdf_file)
mvis_alpha = MechanismVisualizer(mechanism_alpha, URDFVisuals(urdf_file), vis[:alpha])

# Name the joints of the mechanism
# println(bodies(mechanism_alpha))
#  z_prism, roll_joint, pitch_joint, yaw_joint,
base_joint, shoulder_joint, elbow_joint, wrist_joint = joints(mechanism_alpha)
#  virtual_y, virtual_z, virtual_roll, virtual_pitch, 
~, shoulder_body, upper_arm_body, elbow_body, wrist_body = bodies(mechanism_alpha)
# num_virtual_links = 4

shoulder_frame = default_frame(shoulder_body)
upper_arm_frame = default_frame(upper_arm_body)
elbow_frame = default_frame(elbow_body)
wrist_frame = default_frame(wrist_body)
base_frame = root_frame(mechanism_alpha)
# setelement!(mvis_alpha, shoulder_frame)
# # last_link = bodies(mechanism_alpha)[end]
# # body_frame = default_frame(last_link)

frame_names_cob = ["shoulder_cob", "ua_cob", "elbow_cob", "wrist_cob"]
frame_names_com = ["shoulder_com", "ua_com", "elbow_com", "wrist_com"]
cob_vecs = [SVector{3, Float64}([-0.001, -0.003, .032]), SVector{3, Float64}([0.073, 0.0, -0.002]), SVector{3, Float64}([0.015, -0.012, -.003]), SVector{3, Float64}([0.0, 0.0, -.098])]
com_vecs = [SVector{3, Float64}([0.005, -.001, 0.016]), SVector{3, Float64}([0.073, 0.0, 0.0]), SVector{3, Float64}([0.017, -0.026, -0.002]), SVector{3, Float64}([0.0, 0.003, -.098])]
cob_frames = []
com_frames = []
for i in 1:4
    bod = bodies(mechanism_alpha)[i+1]
    frame_cob = CartesianFrame3D(frame_names_cob[i])
    frame_com = CartesianFrame3D(frame_names_com[i])
    cob_vec = cob_vecs[i]
    com_vec = com_vecs[i]
    cob_transform = Transform3D(frame_cob, default_frame(bod), cob_vec)
    com_transform = Transform3D(frame_com, default_frame(bod), com_vec)
    # setelement!(mvis_alpha, default_frame(bod))
    if !(is_fixed_to_body(bod, frame_cob))
        add_frame!(bod, cob_transform)
        push!(cob_frames, frame_cob)
        # setelement!(mvis_alpha, frame_cob)    # visualizes COB frames in MeshCat
    end
    if !(is_fixed_to_body(bod, frame_com))
        add_frame!(bod, com_transform)
        push!(com_frames, frame_com)
        # setelement!(mvis_alpha, frame_com)    # visualizes COM frames in MeshCat
    end
end

# alphabase_com_wrt_linkframe = SVector{3, Float64}([-0.075, -0.006, -.003])
# alphabase_com_wrt_linkframe = SVector{3, Float64}([-0.075, -0.0, -.0])
# linkframe_wrt_vehframe = translation(frame_definitions(vehicle_body)[38])
# # IF THE ARM IS ROTATED THIS HAS TO CHANGE!!!!
# alphabase_com_wrt_vehframe = alphabase_com_wrt_linkframe + linkframe_wrt_vehframe
# alphabase_com_frame = CartesianFrame3D("armbase_com_cob")
# com_transform = Transform3D(alphabase_com_frame, default_frame(vehicle_body), alphabase_com_wrt_vehframe)

# if !(is_fixed_to_body(vehicle_body, alphabase_com_frame))
#     add_frame!(vehicle_body, com_transform)
#     push!(cob_frames, alphabase_com_frame)
#     push!(com_frames, alphabase_com_frame)
#     # setelement!(mvis_alpha, alphabase_com_frame)
# end
# print("THIS SHOULD SAY before_r5m_joint5: ")
# println(frame_definitions(vehicle_body)[38].from)
# # println(alphabase_com_wrt_vehframe)

println("URDFs parsed. Visualizers ready. \n")

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8703
└ @ MeshCat /home/hkolano/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


URDFs parsed. Visualizers ready. 



In [147]:
# buoyancy force setup
# ------------------------------------------------------------------------
#                           BUOYANCY SETUP
# ------------------------------------------------------------------------
# f = rho * g * V
# f = 997 (kg/m^3) * 9.81 (m/s^2) * V_in_L *.001 (m^3) = kg m / s^2
# One time setup of buoyancy forces
volumes = [.018, .203, .025, .155] #  shoulder, ua, elbow, wrist,
buoy_force_mags = volumes * 997 * 9.81 * .001
buoy_lin_forces = []
for mag in buoy_force_mags
    lin_force = FreeVector3D(base_frame, [0.0, 0.0, mag])
    push!(buoy_lin_forces, lin_force)
end
print(buoy_lin_forces)

masses = [.194, .429, .115, .333]
grav_forces = masses*9.81
grav_lin_forces = []
for f_g in grav_forces
    lin_force = FreeVector3D(base_frame, [0.0, 0.0, -f_g])
    push!(grav_lin_forces, lin_force)
end
# println(spatial_inertia(vehicle_body))
# println(spatial_inertia(shoulder_body))
# println(spatial_inertia(upper_arm_body))
# println(spatial_inertia(elbow_body))
# println(spatial_inertia(wrist_body))
# println(mass(mechanism_alpha))

Any[FreeVector3D in "world": [0.0, 0.0, 0.17605026], FreeVector3D in "world": [0.0, 0.0, 1.9854557100000003], FreeVector3D in "world": [0.0, 0.0, 0.24451425000000004], FreeVector3D in "world": [0.0, 0.0, 1.5159883500000002]]

In [148]:
# Hydrodynamics calculator function
# ------------------------------------------------------------------------
#                HYDRODYNAMICS (Grav & Buoy) CALCULATOR
# ------------------------------------------------------------------------
function hydro_calc!(hydro_wrenches::Dict{BodyID, Wrench{Float64}}, t, state_alpha::MechanismState)
    buoy_wrenches = []
    grav_wrenches = []
    names = ["cob1", "cob2", "cob3", "cob4"]
    # Iterate through buoyancy for each body
    for i in 1:4
        # Get the body
        bod = bodies(mechanism_alpha)[i+1]
        # Get default frame of te body
        body_default_frame = default_frame(bod)
        # Get transform between the defualt frame and the center of buoyancy
        # TODO: don't use fixed_transform because it's bad for computation time
        def_to_cob = fixed_transform(bod, body_default_frame, cob_frames[i])
        def_to_com = fixed_transform(bod, body_default_frame, com_frames[i])
        # Transform buoyancy force vector to the body's default frame (rotation only)
        buoy_force_trans = transform(state_alpha, buoy_lin_forces[i], body_default_frame)
        grav_force_trans = transform(state_alpha, grav_lin_forces[i], body_default_frame)
        # Make the wrench: the buoyancy force through a point, the center of buoyancy.
        buoy_wrench = Wrench(Point3D(body_default_frame, translation(inv(def_to_cob))), buoy_force_trans)
        grav_wrench = Wrench(Point3D(body_default_frame, translation(inv(def_to_com))), grav_force_trans)
        # Add wrench to buoy_wrenches
        push!(buoy_wrenches, buoy_wrench)
        push!(grav_wrenches, grav_wrench)
        # Add the buoyancy wrench and grav wrench together
        wrench = buoy_wrench + grav_wrench
        # println(wrench)
        # setelement!(mvis_alpha, Point3D(body_default_frame, translation(inv(def_to_cob))), 0.02, "name")
        # Transform the wrench to the root frame and assign it to the body
        hydro_wrenches[BodyID(bod)] = transform(state_alpha, wrench, root_frame(mechanism_alpha))
    end
end;


In [204]:
function get_des_acc!(des_acc, t, state_alpha::MechanismState, vdot)
    # TODO: implement a simple pt a to pt b trajectory generation
    des_acc[base_joint][1] = 0.0
    des_acc[shoulder_joint][1] = 0.0
    des_acc[elbow_joint][1] = 0.0 #vdot[elbow_joint][1]
    des_acc[wrist_joint][1] = 0.00
end;

In [205]:
# Controller function
# ------------------------------------------------------------------------
#                              CONTROLLER
# ------------------------------------------------------------------------

function simple_control!(torques::AbstractVector, t, state_alpha::MechanismState, hydro_wrenches::Dict{BodyID, Wrench{Float64}}, vdot)
    # Get buoyancy buoyancy forces 
    # hydro_wrenches = Dict{BodyID, Wrench{Float64}}()
    # hydro_calc!(hydro_wrenches, t, state_alpha) 
    # Calculate inverse dynamics of alpha arm (including buoyancy)
    # println("Current acceleration:")
    # println(vdot)
    get_des_acc!(des_acc, t, state_alpha, vdot)
    tau = inverse_dynamics(state_alpha, des_acc, hydro_wrenches)

    # println("Control Tau:")
    # println(tau)

    # Set torques accordingly (controller)
    # torques[velocity_range(state_alpha, yaw_joint)] .= 0.0 
    # torques[velocity_range(state_alpha, shoulder_joint)] .= 0.001 
    # torques[velocity_range(state_alpha, base_joint)] .= 0.001
    # torques[velocity_range(state_alpha, elbow_joint)] .= 0.001
    # torques[velocity_range(state_alpha, wrist_joint)] .= 0.001
    torques[velocity_range(state_alpha, base_joint)] .= tau[base_joint][1]
    torques[velocity_range(state_alpha, shoulder_joint)] .= tau[shoulder_joint][1]
    torques[velocity_range(state_alpha, elbow_joint)] .= 0 #tau[elbow_joint][1]
    torques[velocity_range(state_alpha, wrist_joint)] .= tau[wrist_joint][1]

end;

In [206]:
for joint in joints(mechanism_alpha)
    println(joint.effort_bounds)
end
# new_bounds = similar(base_joint.effort_bounds)
# shoulder_joint.effort_bounds = new_bounds
# shoulder

RigidBodyDynamics.Bounds{Float64}[(-9.0, 9.0)]
RigidBodyDynamics.Bounds{Float64}[(-9.0, 9.0)]
RigidBodyDynamics.Bounds{Float64}[(-9.0, 9.0)]
RigidBodyDynamics.Bounds{Float64}[(-5.0, 5.0)]


In [207]:
# initial conditions
# ------------------------------------------------------------------------
#                          INITIAL CONDITIONS
# ------------------------------------------------------------------------
# Initialize state of alpha arm
state_alpha = MechanismState(mechanism_alpha)
zero!(state_alpha)
set_configuration!(state_alpha, base_joint, 3.141)
# set_configuration!(state, shoulder_joint, -.27079)
# set_configuration!(state, elbow_joint, 3.1415/2+1.3)
# set_configuration!(state, wrist_joint, 0.5)

# Define desired accelerations
des_acc = similar(velocity(state_alpha))
# des_acc[yaw_joint][1] = 0.
des_acc[base_joint][1] = 0.0
des_acc[shoulder_joint][1] = 0.0
des_acc[elbow_joint][1] = 0.0
des_acc[wrist_joint][1] = 0.0


0.0

In [208]:
# simulation
# ------------------------------------------------------------------------
#                              SIMULATION
# ------------------------------------------------------------------------
final_time = 2
ts, qs, vs = simulate_with_ext_forces(state_alpha, final_time, hydro_calc!, simple_control!; Δt = 5e-3)
# ts, qs, vs = simulate(state_alpha, final_time, simple_control!; Δt = 1e-2)

println("Simulation finished.")

Current acceleration:
[0.0, 0.0, 0.0, 0.0]
Current acceleration:
[-0.10799509754134168, -2.865657648699148, -7.432055819876227, -22.45005179353274]
Current acceleration:
[-0.10799529503574182, -2.8656628892272016, -7.432069411135824, -22.450092848725177]
Current acceleration:
[-0.10801478382532377, -2.865646351837529, -7.43212987064325, -22.452560795412346]
Current acceleration:
[-0.10803486316349879, -2.8656455332710777, -7.432231108444836, -22.45515155302142]
Current acceleration:
[-0.10803486463532465, -2.8656455302727473, -7.432231103029536, -22.455151694237564]
Current acceleration:
[-0.10807482172549113, -2.8656386156170592, -7.432419929284902, -22.460291146727016]
Current acceleration:
[-0.10809430680044825, -2.865622065656931, -7.432480406579168, -22.462758070677776]
Current acceleration:
[-0.10815413400420616, -2.8656090164951866, -7.432756839497635, -22.470443972520606]
Current acceleration:
[-0.10815413195553454, -2.8656090080685646, -7.432756814519553, -22.47044372295554]
C


[-0.12940496980380534, -2.8524378071590117, -7.5247888028933785, -25.128836731332466]
Current acceleration:
[-0.1302051469495899, -2.8515456359563354, -7.528092724979418, -25.226941655432977]
Current acceleration:
[-0.13022288817197342, -2.8515211207167748, -7.528149923841815, -25.22906269197811]
Current acceleration:
[-0.13102978923394748, -2.8505752955966295, -7.531456874816542, -25.327887864357507]
Current acceleration:
[-0.1310297334801503, -2.850575221284097, -7.531456538963992, -25.32788209565785]
Current acceleration:
[-0.13184245316303037, -2.84957327296853, -7.534760070287555, -25.427327107444857]
Current acceleration:
[-0.13185984673756646, -2.8495475189810757, -7.534815397325347, -25.42940018037214]
Current acceleration:
[-0.1326774284779428, -2.8484860757159858, -7.5381088740153634, -25.52934760609385]
Current acceleration:
[-0.1326773731635617, -2.8484860033095223, -7.5381085384151, -25.529341911752592]
Current acceleration:
[-0.13349884180518104, -2.847362431332683, -7.5


[-0.14914366948446206, -2.671877328920094, -7.486880986076208, -27.745870033212505]
Current acceleration:
[-0.14913557618783127, -2.6717981904254353, -7.486787667912657, -27.745066337124218]
Current acceleration:
[-0.14830631138433656, -2.6635595436930966, -7.475539161959587, -27.659956040268717]
Current acceleration:
[-0.148306297099914, -2.663559421002367, -7.4755389871187905, -27.659954422025045]
Current acceleration:
[-0.14739394444924614, -2.6550971625014608, -7.463592483667121, -27.565136162948253]
Current acceleration:
[-0.1473843365257103, -2.655017529848982, -7.463493046118847, -27.564163750410323]
Current acceleration:
[-0.14638709035764094, -2.646334523615231, -7.450835342316198, -27.45935640896365]
Current acceleration:
[-0.1463870696587106, -2.646334377672246, -7.450835129847759, -27.45935406594972]
Current acceleration:
[-0.14530465870250298, -2.6374334979342455, -7.437458592223052, -27.34444054498305]
Current acceleration:
[-0.14529360650249715, -2.637353998045466, -7.4


Current acceleration:
[-0.03787129425121687, -2.20266223106521, -6.329071703426479, -12.427330494847444]
Current acceleration:
[-0.03538247370137229, -2.1975202800450244, -6.304231859769372, -11.936819222135775]
Current acceleration:
[-0.035382239153941425, -2.1975197450941595, -6.304229546545196, -11.936773978917374]
Current acceleration:
[-0.03295972228191195, -2.1928424938773983, -6.280165540099872, -11.444019434296242]
Current acceleration:
[-0.032953601393879664, -2.1929432015330805, -6.280410140916204, -11.44292368835269]
Current acceleration:
[-0.03060227703834638, -2.1887487610020555, -6.257192009820174, -10.948376778208265]
Current acceleration:
[-0.030602053781769393, -2.18874825688122, -6.257189736656937, -10.948330873728795]
Current acceleration:
[-0.028324216614532792, -2.1850418021568454, -6.234850076707054, -10.45204689019863]
Current acceleration:
[-0.0283191199986513, -2.185151375434517, -6.235116670320311, -10.45105697562622]
Current acceleration:
[-0.026119237592252

, -2.557304412091708, -6.58496461159814, 11.404016236633353]
Current acceleration:
[-0.02880843931970055, -2.576428917007765, -6.624630865595682, 11.883313163936906]
Current acceleration:
[-0.028808676882818046, -2.5764293015299264, -6.624632327177033, 11.883355999352881]
Current acceleration:
[-0.0311639926994638, -2.595898092659905, -6.665442108857714, 12.36041946418009]
Current acceleration:
[-0.031155144639848413, -2.5960525752969184, -6.6657692059656055, 12.359429215127527]
Current acceleration:
[-0.03359356914537331, -2.61585187982042, -6.707673037733235, 12.833856357844411]
Current acceleration:
[-0.03359382430092967, -2.6158522438357594, -6.7076745367373904, 12.833898466734118]
Current acceleration:
[-0.03611430586951654, -2.635974115071806, -6.750645946408927, 13.305623733616157]
Current acceleration:
[-0.03610400829616178, -2.6361249833713787, -6.750957548347069, 13.304537392031664]
Current acceleration:
[-0.038702380273685145, -2.656555850569094, -6.794944713604275, 13.77313

...

 10.970013312271075, 18.744238171506012]
Current acceleration:
[-0.26169202193628444, 4.942076178073386, 10.970014445102473, 18.744268340928258]
Current acceleration:
[-0.26339138634543907, 4.949675284852108, 10.991505245979532, 19.04488059456883]
Current acceleration:
[-0.2633733250146973, 4.94992606540501, 10.991969562303067, 19.042251161668045]
Current acceleration:
[-0.26493880222024413, 4.957860640848045, 11.01415205509532, 19.338892139497762]
Current acceleration:
[-0.2649389508642249, 4.957860897781068, 11.014153089809843, 19.33892113091591]
Current acceleration:
[-0.2663722154452631, 4.9661294394084425, 11.037023008688337, 19.631681433927593]
Current acceleration:
[-0.2663560171453701, 4.966378145490912, 11.037480746022055, 19.62900925413749]
Current acceleration:
[-0.2676599218083912, 4.974976005783046, 11.061022147518093, 19.917769051420347]
Current acceleration:
[-0.2676600419928301, 4.9749762161015525, 11.06102307973699, 19.91779686165485]
Current acceleration:
[-0.26883641


[-0.24641479004845881, 5.771779515043773, 12.928247175719413, 29.320761912193323]
Current acceleration:
[-0.24641464209180092, 5.771778324691863, 12.928244942484904, 29.320770104645238]
Current acceleration:
[-0.24553400302041584, 5.800254121684895, 12.990802798341349, 29.464052356383284]
Current acceleration:
[-0.2455772646151914, 5.8006300659733885, 12.991509059833433, 29.461580007518226]
Current acceleration:
[-0.24473086724136672, 5.829786064596171, 13.055435822836522, 29.60324788435496]
Current acceleration:
[-0.24473071856220166, 5.829784784142689, 13.055433395555589, 29.60325567287237]
Current acceleration:
[-0.2439194898980851, 5.859627398801366, 13.12074391775665, 29.743374465820985]
Current acceleration:
[-0.2439646664948943, 5.860017509544853, 13.121480098087506, 29.740927222148645]
Current acceleration:
[-0.24319227511569713, 5.890575647778816, 13.188235074556324, 29.879540208131093]
Current acceleration:
[-0.24319212606625298, 5.890574272015097, 13.188232442984948, 29.879


[-0.27026322759622, 8.528915791420532, 18.778641454251076, 34.488170906584294]
Current acceleration:
[-0.27026320860704217, 8.528908258304979, 18.77862587355678, 34.488170960183496]
Current acceleration:
[-0.27208068555629417, 8.626768558629326, 18.983112832756493, 34.517670605393874]
Current acceleration:
[-0.2721355141553563, 8.62808769533579, 18.98578937818023, 34.51465362792045]
Current acceleration:
[-0.27397300366947314, 8.728641660566243, 19.195777166203996, 34.53764193307607]
Current acceleration:
[-0.27397299996539054, 8.728633458012924, 19.19576017089156, 34.537641071443865]
Current acceleration:
[-0.27582658761051077, 8.831898713028918, 19.411280571283406, 34.55378280354475]
Current acceleration:
[-0.27587963714033825, 8.833309220213657, 19.414147644063444, 34.55067116771707]
Current acceleration:
[-0.2777433625174686, 8.939449232995745, 19.63553587190969, 34.559531567950835]
Current acceleration:
[-0.2777433746794792, 8.93944028845727, 19.635517307133007, 34.55952962754717

, 16.92447077677468]
Current acceleration:
[-0.20800287557716077, 19.918461581097095, 42.30524449401538, 15.809306920870169]
Current acceleration:
[-0.20800167848897808, 19.918345759416052, 42.30500082524259, 15.809159453688242]
Current acceleration:
[-0.19785428765491656, 20.38839540248182, 43.27488909079546, 14.645311709346869]
Current acceleration:
[-0.19778380243236882, 20.399449412230595, 43.297966443482316, 14.635960542164364]
Current acceleration:
[-0.1870228901781208, 20.885261218512706, 44.300600061231556, 13.422671740462173]
Current acceleration:
[-0.18702147878904962, 20.885127698904196, 44.300319376039226, 13.422513904374433]
Current acceleration:
[-0.17563916675675387, 21.385897594596216, 45.334029669843424, 12.159047249767777]
Current acceleration:
[-0.17555895647674413, 21.39791728668752, 45.35912334001509, 12.148808421782329]
Current acceleration:
[-0.1635489414438755, 21.914539476125288, 46.4258003918273, 10.834701753905085]
Current acceleration:
[-0.1635473124399732, 

, 14.854613387078743]
Current acceleration:
[-0.12894508523732917, -12.78829386450391, -26.89819863981841, 14.799055504561688]
Current acceleration:
[-0.1487167422542798, -14.67417208528566, -30.874703232948374, 17.094752234278722]
Current acceleration:
[-0.14873476180708012, -14.675993268407865, -30.878535998426006, 17.096769400872393]
Current acceleration:
[-0.16646910350816527, -16.342356393180932, -34.396316371205195, 19.169799652834183]
Current acceleration:
[-0.16599844019606597, -16.29573223954383, -34.298168958210326, 19.11625477279542]
Current acceleration:
[-0.18166830635399797, -17.741143771659665, -37.353914427091695, 20.963985647002378]
Current acceleration:
[-0.18168320430116336, -17.742631711684126, -37.35704880625925, 20.965641644086304]
Current acceleration:
[-0.19536801384085034, -18.975383978724498, -39.96797154428571, 22.596832033534344]
Current acceleration:
[-0.19495891706614563, -18.93522949829101, -39.88336019239114, 22.550598508303004]
Current acceleration:
[-0

In [211]:
# ------------------------------------------------------------------------
#                             VISUALIZATION
# ------------------------------------------------------------------------
# vis = Visualizer()
# open(vis)

# delete!(vis)
# mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf_file))
# set_configuration!(mvis, [0.0, 0.0])
# open(mvis)
MeshCatMechanisms.animate(mvis_alpha, ts, qs; realtimerate = 0.5)
print(last(qs))

# animation = Animation(mvis, ts, qs)
# setanimation!(mvis, animation)

println("\n done.")

[2.969636658252493, -0.27649349467517986, -2.180756955622508, -4.4450903281017595]
 done.
